#### Load all packages

In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import warnings
import datetime
%matplotlib inline
warnings.filterwarnings('ignore')

#### Load data for origin and destination per hour

In [2]:
origin = pd.read_csv('../../Data/20190403/Datalab_Reis_Herkomst_Uur_20190403.csv', sep=';')
origin = origin.dropna()
destination = pd.read_csv('../../Data/20190403/Datalab_Reis_Bestemming_Uur_20190402.csv', sep=';')
destination = destination.dropna()

In [3]:
# List of all stations in the sub-network
stations = ['Amsteldijk','Amstelstation','Amstelveenseweg','Beethovenstraat','Bullewijk','Burg.de Vlugtlaan', 
    'Centraal Station','Cornelis Troostplein','Dam','De Boelelaan/VU','De Pijp','Europaplein',
    'Gaasperplas','Gein','Gerrit v.d. Veenstraat','Heemstedestraat','Henk Sneevlietweg','IJsbaanpad', 
    'Isolatorweg','Jan v.Galenstraat','Keizersgracht','Koningsplein','Kraaienneststation','Leidseplein',
    'Maasstraat','Marie Heinekenplein','Minervaplein','Muntplein','Museumplein','Nieuwezijds Kolk',
    'Nieuwmarkt','Noord','Noorderpark','Olympiaplein','Olympiaweg','Olympisch stadion','Overamstel', 
    'Postjesweg','Prinsengracht','Reigersbos','Rijksmuseum','Roelof Hartplein','Rokin','Scheldestraat',
    'Spaklerweg','Spui','Station Bijlmer ArenA','Station Diemen-Zuid','Station Duivendrecht',
    'Station Ganzenhoef','Station Holendrecht','Station Lelylaan','Station RAI','Station Sloterdijk', 
    'Strandvliet','Van Baerlestraat','Van der Madeweg','Venserpolder','Verrijn Stuartweg','Victorieplein', 
    'Vijzelgracht','VU medisch centrum','Waalstraat','Waterlooplein','Weesperplein','Wibautstraat','Station Zuid']

#### Change dates to datetime objects and add weekday column

In [4]:
origin['Datum'] = origin['Datum'].apply(lambda x: datetime.datetime.strptime(x[:-3], '%m/%d/%Y %H:%M:%S'))
origin['Weekdag'] = origin['Datum'].apply(lambda x: x.isoweekday())

destination['Datum'] = destination['Datum'].apply(lambda x: datetime.datetime.strptime(x[:-3], '%m/%d/%Y %H:%M:%S'))
destination['Weekdag'] = destination['Datum'].apply(lambda x: x.isoweekday())

#### Remove data where the Noord/Zuid line did not run yet
The Noord/Zuid line was opened on the 21st of July, so all data before August is not used.

In [11]:
origin = origin[origin['Datum'] >= datetime.datetime(2018, 8, 1)]
destination = destination[destination['Datum'] >= datetime.datetime(2018, 8, 1)]

#### Apply necessary filters for analysis
The first analysis is only applied on one type of day (1 = Mondays) and for the stations that are included in the sub-network.

In [19]:
origin = origin[(origin['VertrekHalteNaam'].isin(stations)) & (origin['Weekdag'] == 1)]
destination = destination[(destination['AankomstHalteNaam'].isin(stations)) & (destination['Weekdag'] == 1)]

#### Group by hour for further analysis

In [35]:
grouped = origin.groupby(['UurgroepOmschrijving (van vertrek)'])

In [38]:
grouped.describe()

AantalReizen                                \
                                          count        mean         std   min   
UurgroepOmschrijving (van vertrek)                                              
00:00 - 00:59                            1407.0   51.813788   73.899037  10.0   
01:00 - 01:59                             298.0   32.523490   20.622659  10.0   
02:00 - 02:59                             232.0   26.090517   16.975549  10.0   
03:00 - 03:59                             168.0   20.642857   10.671068  10.0   
04:00 - 04:59                              92.0   14.597826    5.785658  10.0   
05:00 - 05:59                            1022.0   33.963796   28.588979  10.0   
06:00 - 06:59                            2166.0   82.951524  105.927924  10.0   
07:00 - 07:59                            4041.0  151.574858  294.714150  10.0   
08:00 - 08:59                            5223.0  226.263259  547.744224  10.0   
09:00 - 09:59                            4848.0  150.418523  347.203969  10.0   
10:00 - 10:59                            4773.0  121.360989  243.960104  10.0   
11:00 - 11:59                            5010.0  111.076447  200.233775  10.0   
12:00 - 12:59                            5295.0  125.091596  227.566928  10.0   
13:00 - 13:59                            5381.0  125.395094  222.063179  10.0   
14:00 - 14:59                            5498.0  136.578756  241.702619  10.0   
15:00 - 15:59                            5609.0  156.329827  272.370645  10.0   
16:00 - 16:59                            5629.0  189.103038  348.672918  10.0   
17:00 - 17:59                            5669.0  246.513671  495.257988  10.0   
18:00 - 18:59                            5379.0  179.126418  350.418589  10.0   
19:00 - 19:59                            4722.0  117.316391  208.933595  10.0   
20:00 - 20:59                            4154.0   92.175012  155.561648  10.0   
21:00 - 21:59                            3845.0   88.118336  148.863813  10.0   
22:00 - 22:59                            3519.0   87.675760  152.031795  10.0   
23:00 - 23:59                            2932.0   70.305252  123.323857  10.0   

                                                               VertrekLat  \
                                     25%   50%     75%     max      count   
UurgroepOmschrijving (van vertrek)                                          
00:00 - 00:59                       14.0  28.0   58.00   972.0     1407.0   
01:00 - 01:59                       16.0  27.0   43.00   117.0      298.0   
02:00 - 02:59                       15.0  20.5   32.00   123.0      232.0   
03:00 - 03:59                       13.0  18.0   25.25    70.0      168.0   
04:00 - 04:59                       11.0  13.0   15.25    43.0       92.0   
05:00 - 05:59                       13.0  22.0   44.00   169.0     1022.0   
06:00 - 06:59                       15.0  41.5  110.75   676.0     2166.0   
07:00 - 07:59                       18.0  37.0  147.00  2231.0     4041.0   
08:00 - 08:59                       21.0  44.0  171.50  5538.0     5223.0   
09:00 - 09:59                       20.0  41.0  133.00  3981.0     4848.0   
10:00 - 10:59                       20.0  41.0  121.00  2793.0     4773.0   
11:00 - 11:59                       20.0  41.0  113.00  1880.0     5010.0   
12:00 - 12:59                       23.0  47.0  119.00  2106.0     5295.0   
13:00 - 13:59                       23.0  50.0  120.00  1842.0     5381.0   
14:00 - 14:59                       24.0  53.0  133.00  2126.0     5498.0   
15:00 - 15:59                       26.0  61.0  151.00  2778.0     5609.0   
16:00 - 16:59                       26.0  69.0  177.00  3408.0     5629.0   
17:00 - 17:59                       28.0  80.0  208.00  3861.0     5669.0   
18:00 - 18:59                       25.0  67.0  161.00  3116.0     5379.0   
19:00 - 19:59                       22.0  51.0  113.00  2349.0     4722.0   
20:00 - 20:59                       20.0  40.0   88.00  1661.0     4154.0   

In [57]:
# Mean is too high, add station with a value of zero if it does not appear in the row? sum/34
origin.groupby(['UurgroepOmschrijving (van vertrek)', 'VertrekHalteNaam']).sum()

VertrekLat  \
UurgroepOmschrijving (van vertrek) VertrekHalteNaam                    
00:00 - 00:59                      Amsteldijk               4.911241   
                                   Amstelstation          186.872385   
                                   Amstelveenseweg         19.429914   
                                   Bullewijk               49.523358   
                                   Burg.de Vlugtlaan       19.352364   
                                   Centraal Station       882.137964   
                                   Dam                    474.560833   
                                   De Pijp                220.088704   
                                   Europaplein             83.153533   
                                   Gaasperplas              4.984686   
                                   Heemstedestraat         14.502636   
                                   Henk Sneevlietweg       24.171966   
                                   Isolatorweg             63.051752   
                                   Jan v.Galenstraat       53.184171   
                                   Keizersgracht           53.752469   
                                   Koningsplein           234.663680   
                                   Kraaienneststation      24.898173   
                                   Leidseplein            644.416133   
                                   Muntplein               83.183216   
                                   Museumplein             34.157711   
                                   Nieuwezijds Kolk       166.357486   
                                   Nieuwmarkt             166.642121   
                                   Noord                  157.799418   
                                   Noorderpark            152.454626   
                                   Overamstel               4.918154   
                                   Prinsengracht          254.007369   
                                   Reigersbos               9.949475   
                                   Rijksmuseum             39.068179   
                                   Rokin                  362.024259   
                                   Spaklerweg              59.050277   
...                                                              ...   
23:00 - 23:59                      Olympiaweg              24.293686   
                                   Olympisch stadion        9.713929   
                                   Overamstel             162.299069   
                                   Postjesweg              87.067344   
                                   Prinsengracht          322.403769   
                                   Reigersbos             159.191593   
                                   Rijksmuseum            263.706104   
                                   Roelof Hartplein       205.058885   
                                   Rokin                  450.085361   
                                   Spaklerweg             162.388261   
                                   Spui                   303.136189   
                                   Station Bijlmer ArenA  628.313045   
                                   Station Diemen-Zuid    158.628247   
                                   Station Duivendrecht   157.977097   
                                   Station Ganzenhoef     164.116715   
                                   Station Holendrecht    158.727009   
                                   Station Lelylaan       454.417632   
                                   Station RAI            166.254752   
                                   Station Sloterdijk     716.155295   
                                   Station Zuid           511.755169   
                                   Strandvliet            133.421261   
                                   VU medisch centrum       9.721031   
                                   Van Baerlestraat       282.997421   
                          

In [58]:
origin

,Datum,UurgroepOmschrijving (van vertrek),VertrekHalteCode,VertrekHalteNaam,VertrekLat,VertrekLon,AantalReizen,Weekdag
1734117,2018-08-06 12:00:00,00:00 - 00:59,00560,Station Bijlmer ArenA,4.947479,52.311258,13,1
1734118,2018-08-06 12:00:00,00:00 - 00:59,00561,Station Bijlmer ArenA,4.947419,52.311329,14,1
1734120,2018-08-06 12:00:00,00:00 - 00:59,02361,Station Sloterdijk,4.839319,52.388335,11,1
1734121,2018-08-06 12:00:00,00:00 - 00:59,05011,Centraal Station,4.899218,52.378108,141,1
1734122,2018-08-06 12:00:00,00:00 - 00:59,05022,Centraal Station,4.901043,52.377720,194,1
1734123,2018-08-06 12:00:00,00:00 - 00:59,05031,Dam,4.893484,52.373411,112,1
1734124,2018-08-06 12:00:00,00:00 - 00:59,05032,Dam,4.895494,52.374929,20,1
1734125,2018-08-06 12:00:00,00:00 - 00:59,05035,Rokin,4.892104,52.369396,45,1
1734126,2018-08-06 12:00:00,00:00 - 00:59,05036,Rokin,4.892305,52.369855,12,1
1734129,2018-08-06 12:00:00,00:00 - 00:59,05052,Waterlooplein,4.902347,52.367273,15,1
